In [3]:
import pprint
from apps.patterns.models import OcurrenceReport
rep = OcurrenceReport.objects.filter(app_name="catalog").order_by('commited_epoch').last()
pp = pprint.PrettyPrinter(width=50, compact=False, indent=2)

print(rep)

acc = {}
for hit in rep.ocurrences:
    parts = hit.split("/")
    app_name = parts[1]
    if app_name in acc:
        element = acc[app_name]
        element['hits'].append(hit)
        element['count'] += 1
        acc[app_name] = element
    else:
        acc[app_name] = { 'hits': [hit], 'count': 1 }

count_acc = list(map(lambda k: (k , acc[k]['count']), acc))
ordered_acc = sorted(sorted(count_acc, key=lambda t: t[0]), key=lambda t: t[1], reverse=True)

if True:
    for name, _count in ordered_acc:
        print(f"{name}")

    for _name, count in ordered_acc:
        print(f"{count}")
else:
    pp.pprint(ordered_acc)

<17f17a592899ac6a75194aa6a9f1532dcc13cbff App:catalog Count:132 2021-02-01 02:01:00>
orders
api
city_management
supply
estimator
catalog_review
catalog_stats
promotions_engine
metrics
adengine
reports
seo
stores_cashiers
brandcenter
catalog_sync_api
stores_reviews
bizdev
brandcenter_api
capacity
cornershop_payment_authorizers
geo
invoices
shopper_commissions
user_store
27
15
13
11
8
7
6
6
5
4
4
4
4
3
3
3
2
1
1
1
1
1
1
1


In [99]:
import os
import ast
import pprint
from typing import List

from apps.patterns.analyzer import BaseFileAnalyzer
from apps.structure import get_full_path

class UsageAnalyzer(BaseFileAnalyzer):
    def __init__(self, *args, **kwargs):
        self.builtins = kwargs.pop('builtins', [])
        self.match_against = kwargs.pop('match_against', [])
        self.verbose = kwargs.pop('verbose', False)
        self.usages = []
        super(UsageAnalyzer, self).__init__(*args, **kwargs)
    
    def visit_FunctionDef(self, node):
        if self.verbose: print(f"** --> Function: {node.name} at {node.lineno}")
        for stmt in node.body:
            self.build_command(stmt)
                    
    def record_usage(self, deconstructed_usage: List[str], lineno: int):
        deconstructed_usage.reverse()
        usage = ".".join(deconstructed_usage)
        usage += f":{str(lineno)}"
        self.usages.append(usage)
        
    def get_analysis(self):
        return self.usages
                
    def build_command(self, node, acc=[]):
        # TODO check apps/subscriptions/process.py
        if self.verbose: print(f"{node.__class__}:{node.lineno}")
            
        if node.__class__ == ast.Return:
            if node.value is not None:
                acc = self.build_command(node.value, acc)
            
        if node.__class__ == ast.With:
            for stmt in node.body:
                acc = self.build_command(stmt, acc)
        
        if node.__class__ == ast.Tuple:
            for expr in node.elts:
                acc = self.build_command(expr, acc)
                
        if node.__class__ == ast.Expr:
            acc = self.build_command(node.value, acc)
            
        if node.__class__ == ast.For:
            acc = self.build_command(node.iter, acc)
            for stmt in node.body:
                acc = self.build_command(stmt, acc)
            for stmt in node.orelse:
                acc = self.build_command(stmt, acc)

        if node.__class__ == ast.Try:
            for stmt in node.body:
                acc = self.build_command(stmt, acc)
            for stmt in node.handlers:
                acc = self.build_command(stmt, acc)
            for stmt in node.orelse:
                acc = self.build_command(stmt, acc)
            for stmt in node.finalbody:
                acc = self.build_command(stmt, acc)
    
        if node.__class__ == ast.ExceptHandler:
            if node.type is not None:
                acc = self.build_command(node.type, acc)
            for stmt in node.body:
                acc = self.build_command(stmt, acc)
                
        if node.__class__ == ast.Compare:
            acc = self.build_command(node.left, acc)
            for stmt in node.comparators:
                acc = self.build_command(stmt, acc)
                
        if node.__class__ == ast.BoolOp:
            for stmt in node.values:
                acc = self.build_command(stmt, acc)
        
        if node.__class__ == ast.Assign:
            acc = self.build_command(node.value, acc)
            
        if node.__class__ == ast.If:
            acc = self.build_command(node.test, acc)
            for stmt in node.body:
                acc = self.build_command(stmt, acc)
            for stmt in node.orelse:
                acc = self.build_command(stmt, acc)
            
        if node.__class__ == ast.Subscript:
            acc = self.build_command(node.value, acc)
            
        if node.__class__ == ast.Call:
            if node.func.__class__ == ast.Name and node.func.id in self.builtins:
                for inner_node in node.args:
                    acc = self.build_command(inner_node, acc)
            else:
                acc = self.build_command(node.func, acc)
                
        if node.__class__ == ast.Attribute:
            # This if captures things like Order.CONSTANT
            if node.value.__class__ == ast.Name and node.value.id in self.match_against:
                acc.append(node.attr)
                acc.append(node.value.id)

                self.record_usage(acc, lineno=node.lineno)
                acc.clear()
            else:
                acc.append(node.attr)
                acc = self.build_command(node.value, acc)
        
        if node.__class__ == ast.Dict:
            for dict_value in node.values:
                acc = self.build_command(dict_value, acc)
                
        if node.__class__ == ast.Constant:
            acc.append(node.value)
            
        if node.__class__ == ast.Name: 
            if node.id in self.match_against:
                acc.append(node.id)
                self.record_usage(acc, lineno=node.lineno)
                
            acc.clear()
        return acc

    
class ModelsAnalyzer(BaseFileAnalyzer):
    def __init__(self, *args, **kwargs):
        self.model_names = []
        super(ModelsAnalyzer, self).__init__(*args, **kwargs)
        
    def get_analysis(self):
        return self.model_names
    
    def visit_ClassDef(self, node):
        self.model_names.append(node.name)
            
            
orders_models_anlyz = ModelsAnalyzer(entrypoint="/Users/seba.gonzalez/repo/cornershop-static-analysis/repo/apps/orders/models.py")
orders_models_anlyz.start()
model_names = orders_models_anlyz.get_analysis()

pp = pprint.PrettyPrinter(width=21, compact=True, indent=4)
for hit in acc["carts"]["hits"]:   
    file, lineno = hit.split(":")
    file_path = get_full_path(file_path=file)
    print(file_path)
    print(lineno)
    config = dict(
        verbose=False,
        match_against=model_names,
        builtins=['list']
    )

    anlyz = UsageAnalyzer(entrypoint=file_path, **config)
    anlyz.start()
    pp.pprint(anlyz.get_analysis())

/Users/seba.gonzalez/repo/cornershop-static-analysis/repo/apps/carts/firebase.py
3
[   'Cart.objects.get:35']
/Users/seba.gonzalez/repo/cornershop-static-analysis/repo/apps/carts/views.py
10
[   'Cart.objects.get.format.format:22']
